# Underwater Imagenet Implementation

### Importing the library

In [1]:
from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten,Dropout,Input
from keras.layers import MaxPooling2D,AveragePooling2D,Concatenate

### Defining the Inception Module

In [2]:
def inception(x,n,i):
    '''
    The kernel initializer, number of filters and the padding was not defined on the paper. 
    So I have taken kernel initializer as he_uniform as it works better for images. 
    And the number of filters here I have taken as 64.
    
    This function basically merges 3 different types of Convolution layer and then applies Maxpooling layer to it
    and then returns.
    '''
    conv5=Conv2D(n,(5,5),padding="same",activation="relu",kernel_initializer="he_uniform")(x)
    conv3=Conv2D(n,(3,3),padding="same",activation="relu",kernel_initializer="he_uniform")(x)
    conv1=Conv2D(n,(1,1),padding="same",activation="relu",kernel_initializer="he_uniform")(x)
    merged=Concatenate(axis=3,name='inception_network_'+str(i))([conv1,conv3,conv5])
    merged=MaxPooling2D(pool_size=(2,2),padding="same")(merged)
    return merged

### Defining the Architecture of the CNN

In [3]:
img_input=Input(shape=(200,200,3))

# the number of filters were not given
# total number of dense connections were also not provided
##<<========= Model Architecture =========>>
x= Conv2D(filters=32,kernel_size=(7,7),strides=(2,2),activation='relu',kernel_initializer="he_uniform")(img_input)
x=MaxPooling2D(3,2,padding="same")(x)
x=Conv2D(32,3,1,activation="relu",kernel_initializer="he_uniform")(x)
x=MaxPooling2D(3,2,padding="same")(x)
x=inception(x,64,1)
x=inception(x,64,2)
x=MaxPooling2D(3,2,padding="same")(x)
x=inception(x,64,3)
x=inception(x,64,4)
x=inception(x,64,5)
x=inception(x,64,6)
x=inception(x,64,7)
x=MaxPooling2D(3,2,padding="same")(x)
x=inception(x,64,8)
x=inception(x,64,9)
x=AveragePooling2D(2,padding="same")(x)
x=Dropout(0.2)(x)
x=Flatten()(x)
x=Dense(units=5,activation='softmax')(x)

In [4]:
model=Model(inputs=img_input,outputs=x)

In [5]:
## compiling the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
## summary of the model
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 97, 97, 32)   4736        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 49, 49, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 47, 47, 32)   9248        max_pooling2d[0][0]              
_______________________________________________________________________________________

In [7]:
# fittig the above model to the images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.3,
                                   zoom_range = 0.3,
                                   horizontal_flip = True,
                                   vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('data/trainA',
                                                 target_size = (200,200),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('data/trainB',
                                            target_size = (200,200),
                                            batch_size = 16,
                                            class_mode = 'categorical')

model.fit_generator(training_set, 
                         steps_per_epoch = 256,
                         epochs = 12,
                         validation_data = test_set,
                         validation_steps = 10)

Found 6128 images belonging to 5 classes.
Found 6127 images belonging to 5 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/12
256/256 [==============================] - 29s 115ms/step - loss: 1.2819 - accuracy: 0.4761 - val_loss: 1.2053 - val_accuracy: 0.4500
Epoch 2/12
256/256 [==============================] - 31s 122ms/step - loss: 1.1891 - accuracy: 0.5278 - val_loss: 1.2159 - val_accuracy: 0.4812
Epoch 3/12
256/256 [==============================] - 31s 122ms/step - loss: 1.1419 - accuracy: 0.5496 - val_loss: 1.3131 - val_accuracy: 0.5063
Epoch 4/12
256/256 [==============================] - 30s 118ms/step - loss: 1.0905 - accuracy: 0.5630 - val_loss: 1.2385 - val_accuracy: 0.4375
Epoch 5/12
256/256 [==============================] - 30s 118ms/step - loss: 1.0563 - accuracy: 0.5750 - val_loss: 1.1089 - val_accuracy: 0.5188
Epoch 6/12
256/256 [==============================] - 32s 125ms/step - loss: 1.0521 - accuracy: 0.5732 - val_loss: 